In [24]:
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse.linalg as splin
import scipy.sparse as sparse
import scipy.sparse.linalg as splg
import json
import pickle


import random
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import ndcg_score as ndcg

# from ignite.metrics.recall import Recall as t_recall
import torch

# data loading

In [2]:
dict_df_rating = dict()

#20 millions

# #links = pd.read_csv('ml-20m/links.csv')
# df_ratings = pd.read_csv('ml-20m/ratings.csv')
# df_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
# df_ratings.head()


#1 million MovieLens 

df_ratings = pd.read_csv('ml-1m/ratings.dat', sep="::", header=None)
df_ratings.columns = ['user_id', 'object_id', 'rating', 'timestamp']
df_ratings.drop('timestamp', axis='columns', inplace=True)
df_ratings.dropna(inplace=True)
df_ratings['user_id'] = df_ratings['user_id'].astype("category").cat.codes
df_ratings['object_id'] = df_ratings['object_id'].astype("category").cat.codes

dict_df_rating['ML1M'] = dict()
dict_df_rating['ML1M']['original'] = df_ratings

#Books http://www2.informatik.uni-freiburg.de/~cziegler/BX/

#df_ratings = pd.read_csv('Books Data/BX-Book-Ratings.csv', sep=';')
df_ratings = pd.read_csv('Books Data/BX-Book-Ratings_short.csv', sep=';')
df_ratings.columns = ['user_id', 'object_id', 'rating']
df_ratings.dropna(inplace=True)
df_ratings['user_id'] = df_ratings['user_id'].astype("category").cat.codes
df_ratings['object_id'] = df_ratings['object_id'].astype("category").cat.codes


dict_df_rating['BX'] = dict()
dict_df_rating['BX']['original'] = df_ratings

#Music  http://jmcauley.ucsd.edu/data/amazon/

#df_ratings = pd.read_csv('Digital music/ratings_Digital_Music.csv', sep=',', header=None)
df_ratings = pd.read_csv('Digital music/ratings_Digital_Music_short.csv', sep=',', header=None)
df_ratings.columns = ['user_id', 'object_id', 'rating', 'timestamp']
df_ratings.drop('timestamp', axis='columns', inplace=True)
df_ratings.dropna(inplace=True)
df_ratings['user_id'] = df_ratings['user_id'].astype("category").cat.codes
df_ratings['object_id'] = df_ratings['object_id'].astype("category").cat.codes


dict_df_rating['DM'] = dict()
dict_df_rating['DM']['original'] = df_ratings

/cephfs/projects/psoker/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
for ds in dict_df_rating:
    df_ratings = dict_df_rating[ds]['original']
    all_cells = df_ratings.object_id.nunique() * df_ratings.user_id.nunique()
    sparcity_rate = len(df_ratings) / all_cells
    print(f'sparcity for {ds} dataset: {sparcity_rate}')

sparcity for ML1M dataset: 0.044683625622312845
sparcity for BX dataset: 0.0022166389175659485
sparcity for DM dataset: 0.0014640480404597422


In [38]:
for ds in dict_df_rating:
    df_ratings = dict_df_rating[ds]['original']
    print(ds, df_ratings.rating.unique())

[5 3 4 2 1]
[ 0  5  6 10  9  7  8  2  4  3  1]
[5. 3. 4. 1. 2.]


In [4]:
# for ds in dict_df_rating:
#     print(ds)
    
#     df_ratings = dict_df_rating[ds]['original'].copy()
    
#     df_ratings['user_id'] =  df_ratings['user_id'].astype('str')
#     df_ratings['object_id'] =  df_ratings['object_id'].astype('str')

#     dense_rating = df_ratings.pivot(index='user_id', columns='object_id', values='rating')
#     dense_rating = dense_rating.fillna(0)
#     dict_df_rating[ds]['dense'] = dense_rating

In [5]:
#forming sparse matrixes

for ds in dict_df_rating:
    print(ds)
    df_ratings = dict_df_rating[ds]['original'].copy()

#     dict_obj = {obj:i for i, obj in enumerate(sorted(df_ratings.object_id.unique()))}
#     dict_user = {obj:i for i, obj in enumerate(sorted(df_ratings.user_id.unique()))}

#     df_ratings['user_id'] = df_ratings['user_id'].astype("category").cat.codes
#     df_ratings['object_id'] = df_ratings['object_id'].astype("category").cat.codes
    
#     dict_df_rating[ds]['original'] = df_ratings
    
    rows = df_ratings['user_id'].astype("int")
    cols = df_ratings['object_id'].astype("int")

    ratings = df_ratings.rating.astype("int")
    
    data_sparse = sparse.csr_matrix((ratings, (rows, cols)), shape=(df_ratings.user_id.nunique(), 
                                                                df_ratings.object_id.nunique()), dtype='int32')
    
    dict_df_rating[ds]['sparse'] = data_sparse

ML1M
BX
DM


In [6]:
df_ratings

,user_id,object_id,rating
0,3082,0,5.0
1,4828,0,5.0
2,2870,0,5.0
3,2194,0,5.0
4,3435,0,3.0
...,...,...,...
59259,3042,8364,5.0
59260,3042,8365,5.0
59261,3290,8366,5.0
59262,1574,8366,4.0


In [7]:
for ds in dict_df_rating:
    print(ds, dict_df_rating[ds]['sparse'].shape)

ML1M (6040, 3706)
BX (11707, 16162)
DM (4838, 8367)


# Methods

In [8]:
def recommend(user_id, data_sparse, user_vecs, item_vecs):
    """Recommend items for a given user given a trained model
    
    Args:
        user_id (int): The id of the user we want to create recommendations for.
        
        data_sparse (csr_matrix): Our original training data.
        
        user_vecs (csr_matrix): The trained user x features vectors
        
        item_vecs (csr_matrix): The trained item x features vectors
        
        item_lookup (pandas.DataFrame): Used to map artist ids to artist names
        
        num_items (int): How many recommendations we want to return:
        
    Returns:
        recommendations (pandas.DataFrame): DataFrame with num_items artist names and scores
    
    """
  
    # Get all interactions by the user
    user_interactions = data_sparse[user_id,:].toarray()

    # We don't want to recommend items the user has consumed. So let's
    # set them all to 0 and the unknowns to 1.
    user_interactions = user_interactions.reshape(-1) + 1 #Reshape to turn into 1D array
    #user_interactions[user_interactions > 1] = 0

    # This is where we calculate the recommendation by taking the 
    # dot-product of the user vectors with the item vectors.
    rec_vector = (user_vecs[user_id,:] @ item_vecs.T).toarray()

    # Let's scale our scores between 0 and 1 to make it all easier to interpret.
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1, 1))[:,0]
    recommend_vector = user_interactions * rec_vector_scaled
   
    # Get all the artist indices in order of recommendations (descending) and
    # select only the top "num_items" items. 
    item_idx = np.argsort(recommend_vector)[::-1]

    objects = []
    scores = []

    # Loop through our recommended artist indicies and look up the actial artist name
    for idx in item_idx:
        objects.append(idx)
        scores.append(recommend_vector[idx])

    # Create a new dataframe with recommended artist names and scores
    recommendations = pd.DataFrame({'object_id': objects, 'score': scores})
    
    return recommendations

In [9]:
def recall(actual, predicted, k):
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / float(len(act_set))
    return result

In [10]:
all_metrix_res_dict = dict()

for ds in dict_df_rating:
    all_metrix_res = pd.DataFrame([], index = ['Recall20', 'Recall50', 'NDCG100'])
    all_metrix_res_dict[ds] = all_metrix_res

## ALS


In [55]:
def als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
    """ Implementation of Alternating Least Squares with implicit data. We iteratively
    compute the user (x_u) and item (y_i)from tor vectors using the following formulas:
 
    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
    y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))
 
    Args:
        sparse_data (csr_matrix): Our sparse user-by-item matrix
 
        alpha_val (int): The rate in which we'll increase our confidence
        in a preference with more interactions.
 
        iterations (int): How many times we alternate between fixing and 
        updating our user and item vectors
 
        lambda_val (float): Regularization value
 
        features (int): How many latent features we want to compute.
    
    Returns:     
        X (csr_matrix): user vectors of size users-by-features
        
        Y (csr_matrix): item vectors of size items-by-features
     """

    assert type(sparse_data) == sparse.csr_matrix, "Matrix should be sparse in format of csr"
    
    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape
    
    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))

    I = sparse.eye(features)
    lI = lambda_val * I
    
    for _ in tqdm(range(iterations)):
        
        # Precompute Y-transpose-Y and X-transpose-X
        yTy = Y.T @ Y
        xTx = X.T @ X

        # Loop through all users
        for u in range(user_size):
            u_row = sparse_data[u,:].toarray()
            #print(((yTy + lI).shape, (u_row @ Y).T.shape))
            X[u] = spsolve(yTy + lI, (u_row @ Y).T)

        for i in range(item_size):
            i_row = sparse_data[:,i].T.toarray()
            Y[i] = spsolve(xTx + lI, (i_row @ X).T)

    return X, Y

In [57]:
als_res_dict = dict()

for ds in dict_df_rating:
    sparse_local = dict_df_rating[ds]['sparse']
    user_vecs, item_vecs = als(sparse_local, iterations=15, features=20, alpha_val=40)
    als_res_dict[ds] = (user_vecs, item_vecs)

In [58]:
with open('als_vectors.pickle', 'wb') as f:
    pickle.dump(als_res_dict, f)


In [17]:
for ds in dict_df_rating:
    df_ratings = dict_df_rating[ds]['original']
    all_users = df_ratings.user_id.unique()
    user_vecs_loc, item_vecs_loc = als_res_dict[ds]
    sparse_local = dict_df_rating[ds]['sparse']
    
    rec20_list = []
    rec50_list = []
    NDCG_list = []
    
    for user_id in tqdm(all_users):
        #print(sparse_local.shape, user_vecs_loc.shape, item_vecs_loc.shape)
        recommendations = recommend(user_id, sparse_local, user_vecs_loc, item_vecs_loc)
        dense_ratings_user = df_ratings[(df_ratings['user_id'] == user_id) & (df_ratings['rating'] > 0)]
        
        compilation = pd.merge(dense_ratings_user, recommendations, how='inner', on = 'object_id')
        #dense_ratings_user.join(recommendations, on='object_id', how='inner')
        compilation['score']  = compilation.score*5
        compilation['score_round'] = round(compilation.score).astype(int)
        
        rec20 = recall(compilation.rating.values, compilation.score_round.values, k=20)
        rec50 = recall(compilation.rating.values, compilation.score_round.values, k=50)
        NDCG = ndcg(compilation.rating.values.reshape((1, -1)), compilation.score.values.reshape((1, -1)), k=100)
        
        rec20_list.append(rec20)
        rec50_list.append(rec50)
        NDCG_list.append(NDCG)
        
    mertix = []
    for lst in [rec20_list, rec50_list, NDCG_list]:
        mertix.append(np.mean(lst))
        
    all_metrix_res_dict[ds]['ALS'] = mertix

KeyboardInterrupt: 

## ALS++

In [ ]:
def alspp(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
    """ Implementation of Alternating Least Squares with implicit data. We iteratively
    compute the user (x_u) and item (y_i)from tor vectors using the following formulas:
 
    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
    y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))
 
    Args:
        sparse_data (csr_matrix): Our sparse user-by-item matrix
 
        alpha_val (int): The rate in which we'll increase our confidence
        in a preference with more interactions.
 
        iterations (int): How many times we alternate between fixing and 
        updating our user and item vectors
 
        lambda_val (float): Regularization value
 
        features (int): How many latent features we want to compute.
    
    Returns:     
        X (csr_matrix): user vectors of size users-by-features
        
        Y (csr_matrix): item vectors of size items-by-features
     """

    assert type(sparse_data) == sparse.csr_matrix, "Matrix should be sparse in format of csr"
    
    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape
    
    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))
    
#     I = sparse.csr_matrix((np.ones(features), (np.ones(features), np.ones(features))), 
#                            shape=(features, features), dtype='int32')
#     #I = sparse.eye(features)
#     lI = lambda_val * I
    
    for _ in tqdm(range(iterations)):
        #print(Y.shape)
        
        # Precompute Y-transpose-Y and X-transpose-X
        ind_rand = list(set(random.choices(list(range(features)), weights=None, k=features//2)))
        Y_rand = Y[:, ind_rand]
        X_rand = X[:, ind_rand]
        
        lI = lambda_val * sparse.eye(len(ind_rand))
        
        yTy = Y_rand.T @ Y_rand
        xTx = X_rand.T @ X_rand

        # Loop through all users
        for u in range(user_size):
            u_row = sparse_data[u,:].toarray()
            X[u, ind_rand] = spsolve(yTy + lI, (u_row @ Y_rand).T)

        for i in range(item_size):
            i_row = sparse_data[:,i].T.toarray()
            Y[i, ind_rand] = spsolve(xTx + lI, (i_row @ X_rand).T)

    return X, Y


In [ ]:
alspp_res_dict = dict()

for ds in dict_df_rating:
    sparse_local = dict_df_rating[ds]['sparse']
    user_vecs, item_vecs = alspp(sparse_local, iterations=15, features=20, alpha_val=40)
    alspp_res_dict[ds] = (user_vecs, item_vecs)

In [ ]:
# with open('alspp_vectors.pickle', 'wb') as f:
#     pickle.dump(alspp_res_dict, f)

In [ ]:
for ds in dict_df_rating:
    df_ratings = dict_df_rating[ds]['original']
    all_users = df_ratings.user_id.unique()
    user_vecs_loc, item_vecs_loc = alspp_res_dict[ds]
    
    rec20_list = []
    rec50_list = []
    NDCG_list = []
    
    for user_id in tqdm(all_users):
        recommendations = recommend(user_id, data_sparse, user_vecs_loc, item_vecs_loc)
        dense_ratings_user = df_ratings[(df_ratings['user_id'] == user_id) & (df_ratings['rating'] > 0)]
        
        compilation = pd.merge(dense_ratings_user, recommendations, how='inner', on = 'object_id')
        #dense_ratings_user.join(recommendations, on='object_id', how='inner')
        compilation['score']  = compilation.score*5
        compilation['score_round'] = round(compilation.score).astype(int)
        
        rec20 = recall(compilation.rating.values, compilation.score_round.values, k=20)
        rec50 = recall(compilation.rating.values, compilation.score_round.values, k=50)
        NDCG = ndcg(compilation.rating.values.reshape((1, -1)), compilation.score.values.reshape((1, -1)), k=100)
        
        rec20_list.append(rec20)
        rec50_list.append(rec50)
        NDCG_list.append(NDCG)
        
    mertix = []
    for lst in [rec20_list, rec50_list, NDCG_list]:
        mertix.append(np.mean(lst))
        
    all_metrix_res_dict[ds]['ALSPP'] = mertix

all_metrix_res

# ALS with bias

In [19]:
def biased_als(data, iterations, lmbda, features):

    user_size, item_size = data.shape
    
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features+1)))
    Y = sparse.csr_matrix(np.random.normal(size = (features+1, item_size)))
    
    beta = np.zeros((1, user_size))
    gamma = np.zeros((1, item_size))
    
    I = sparse.eye(features+1)
    lI = lmbda * I
    
    for _ in tqdm(range(iterations)):
        
        X_wave = X.copy()
        #print(X_wave[:, 0].shape, np.ones((1, user_size)).shape)
        X_wave[:, 0] = np.ones((user_size, 1))
        Y_wave = Y.copy()
        Y_wave[0,:] = gamma
        

        xTx = X_wave.T @ X_wave

        for i in range(item_size):
            
            r_i = data[:, i].T.toarray()
            r_i_beta = r_i - beta
            
#             print('I r i shape', r_i.shape)
#             print('I r i beta shape', r_i_beta.shape)
#             print('I X_wave shape', X_wave.shape)
#             print('I Y_wave shape', Y_wave.shape, '\n')
            
            Y[:,i] = spsolve(xTx + lI, (r_i_beta @ X_wave).T)
            #Y[:,i] = splg.inv(X_wave.T @ X_wave + lmbda * sparse.eye(features+1)) @ X_wave.T @ r_i_beta.T
        
        yTy = Y_wave @ Y_wave.T
        Y_wave = Y.copy()
        Y_wave[0, :] = np.ones((1, item_size))
        X_wave = X.copy()
        X_wave[:,0] = beta    
        for u in range(user_size):
            r_u = data[u, :].toarray()
            r_u_gamma = r_u - gamma
            
#             print('U r u shape', r_u.shape)
#             print('U r u gamma shape', r_u_gamma.shape)
#             print('U X_wave shape', X_wave.shape)
#             print('U Y_wave shape', Y_wave.shape)
#             print('\n\n')

            X[u,:] = spsolve(Y_wave @ Y_wave.T + lI, Y_wave @ r_u_gamma.T)
            #X[u,:] = splg.inv(Y_wave @ Y_wave.T + lmbda*sparse.eye(features+1)) @ Y_wave @ r_u_gamma.T
            
    return X, Y.T

In [ ]:
als_bias_res_dict = dict()

for ds in dict_df_rating:
    sparse_local = dict_df_rating[ds]['sparse']
    user_vecs, item_vecs = biased_als(sparse_local, iterations=15, features=20, lmbda=10)
    als_bias_res_dict[ds] = (user_vecs, item_vecs)

In [ ]:
# with open('als_bias_vectors.pickle', 'wb') as f:
#     pickle.dump(als_bias_res_dict, f)

# Evaluation

In [45]:
def get_evaluation(dict_with_res, method_name):
    for ds in dict_df_rating:
        df_ratings = dict_df_rating[ds]['original']
        all_users = df_ratings.user_id.unique()
        user_vecs_loc, item_vecs_loc = dict_with_res[ds]
        sparse_local = dict_df_rating[ds]['sparse']

        rec20_list = []
        rec50_list = []
        NDCG_list = []

        for user_id in tqdm(all_users):
            recommendations = recommend(user_id, sparse_local, user_vecs_loc, item_vecs_loc)
            dense_ratings_user = df_ratings[(df_ratings['user_id'] == user_id) & (df_ratings['rating'] > 0)]
    
            compilation = pd.merge(dense_ratings_user, recommendations, how='inner', on = 'object_id')
            
            if len(compilation) < 2:
                continue
            #dense_ratings_user.join(recommendations, on='object_id', how='inner')
            compilation['score']  = compilation.score*5
            compilation['score_round'] = round(compilation.score).astype(int)

            rec20 = recall(compilation.rating.values, compilation.score_round.values, k=20)
            rec50 = recall(compilation.rating.values, compilation.score_round.values, k=50)
            NDCG = ndcg(compilation.rating.values.reshape((1, -1)), compilation.score.values.reshape((1, -1)), k=100)

            rec20_list.append(rec20)
            rec50_list.append(rec50)
            NDCG_list.append(NDCG)

        mertix = []
        for lst in [rec20_list, rec50_list, NDCG_list]:
            mertix.append(np.mean(lst))

        all_metrix_res_dict[ds][method_name] = mertix


In [50]:
with open('als_bias_vectors.pickle', 'rb') as f:
    als_bias_res_dict = pickle.load(f)

get_evaluation(als_bias_res_dict, 'ALS_bias')

In [ ]:
with open('als_vectors.pickle', 'rb') as f:
    als_res_dict = pickle.load(f)

get_evaluation(als_res_dict, 'ALS')

In [51]:
with open('alspp_vectors.pickle', 'rb') as f:
    alspp_res_dict = pickle.load(f)

get_evaluation(alspp_res_dict, 'ALSPP')

In [53]:
for ds in dict_df_rating:
    print(ds)
    print(all_metrix_res_dict[ds], end='\n')

ML1M
          ALS_bias     ALSPP
Recall20  0.502870  0.334421
Recall50  0.696623  0.476719
NDCG100   0.967787  0.960765
BX
          ALS_bias     ALSPP
Recall20  0.159582  0.090857
Recall50  0.174330  0.094478
NDCG100   0.982831  0.993770
DM
          ALS_bias     ALSPP
Recall20  0.122520  0.078996
Recall50  0.137759  0.088320
NDCG100   0.991055  0.995574
